# Testing notebook
This is a testing notebook, serving no other purpose other than testing things out while developping.

## Initial implementation

In [ ]:
import sys, os
sys.path.append("..")

import numpy as np
from random import randint

from src.models import OpenAIBot
from src.prompts.game24 import foa_step_prompt, value_prompt, value_last_step_prompt, bfs_prompt
from src.tasks.base import DATA_PATH
from src.tasks.game24 import Game24
from src.methods.agents import Agents
from src.methods.resampler import Resampler

%load_ext autoreload
%autoreload 2

bot = OpenAIBot(model="gpt-3.5-turbo-1106")
#bot = OpenAIBot(model="gpt-4")

In [ ]:
# Prompt testing
from src.prompts.game24 import foa_step_prompt, value_prompt, value_last_step_prompt, bfs_prompt, cot_prompt
x = 20
input = "1 24"

prompt = foa_step_prompt.format(input=input) 

for i in range(x):
    response = bot.request(prompt)

print(f"Prompt \n{prompt}")
print("----\n----\n")
print(f"Response \n{response}")

In [ ]:
# Game24 task step function

task = Game24(bot)
idx = randint(0, len(task))
task.get_input(idx)


value_numbers = []
for i in range(task.max_steps):
    task.step()
    value_number = task.evaluate()
    value_numbers.append(value_number)
summary = []

for i in range(len(task.steps)):
    temp = task.steps[i] + f" [Value : {value_numbers[i]}]"
    summary.append(temp)

print(f"Input: {task.input}")
print("\n".join(summary))

In [ ]:
# Resampler

values = np.array([20, 10, 5, 1, 0.0001])
resampler = Resampler()

# Normalized resampling
draw = resampler.resample(values, resample_method="normalization")
print(f"Normalized : {draw}")


# Greedy
draw = resampler.resample(values, resample_method="greedy")
print(f"Greedy : {draw}")


In [ ]:
# Multiple agents

idx_input = 8
n_evaluations = 2
n_agents = 2

# Create agents
agents = Agents(task=Game24, idx_input=idx_input, n_agents=n_agents, init=False, model=bot, n_evaluations=2)

# Run agents
for i in range(agents.max_steps-1):
    agents.step()
    agents.evaluate()
    agents.resample()

# Final step : Finish answers format + Choose best answer
agents.step() 

# Log results
current_path = os.getcwd()
log_path = os.path.join(os.path.dirname(current_path), "logs")
agents.create_log(repo_path=log_path)

In [ ]:
# Test results funciton

idx_input = 8
n_evaluations = 3
n_agents = 5

# Create agents
agents = Agents(task=Game24, idx_input=idx_input, n_agents=n_agents, model=bot)

# Run agents
for i in range(agents.max_steps-1):
    agents.step()
    agents.evaluate(n=n_evaluations)
    agents.resample()

# Final step : Finish answers format + Choose best answer
agents.step() 

print("\n\n".join(["\n".join(agent.steps) for agent in agents.agents]))
print(f"\nResults : {agents.test_output()}")

## Async Implementation

In [ ]:
import asyncio
# set up logging
import logging
import os
import json
import random
import argparse

import numpy as np
import pandas as pd
from diskcache import Cache
from datetime import datetime
from collections import Counter


# TODO: Not sure if this is correct, I didn't know how else to handle the package paths
import sys
sys.path.append(os.getcwd()) # Project root!!

from async_engine.cached_api import CachedOpenAIAPI
from async_engine.round_robin_manager import AsyncRoundRobin
from async_implementation.agents.gameof24 import GameOf24Agent
from async_implementation.states.gameof24 import GameOf24State
from utils import create_folder, email_notification
from async_engine.mock_batched_async import BatchingAPI

%load_ext autoreload
%autoreload 2

### Resampling

In [ ]:
# Resampling methods

values = [0, 0.001, 0.003, 1, 3, 20, 60]

print(f"Linear probabilities : {[round(p, 3) for p in linear(values)]}")
print(f"Logistic probabilities : {[round(p, 3) for p in logistic(values)]}")

### Hash function

In [ ]:
# Gameof24State Hash

path = 'data/24_tot.csv'
data = pd.read_csv(path).Puzzles.tolist()

puzzle_idx = 0
puzzle = data[puzzle_idx]

num_agents = 1

s1 = GameOf24State(puzzle=puzzle, current_state=puzzle, steps=[], randomness=random.randint(0, 1000))
s2 = GameOf24State(puzzle=puzzle, current_state=puzzle, steps=[], randomness=random.randint(0, 1000))
s3 = GameOf24State(puzzle=puzzle, current_state=puzzle, steps=["something"], randomness=random.randint(0, 1000))
s4 = GameOf24State(puzzle=puzzle, current_state="24", steps=["something"], randomness=random.randint(0, 1000))

# Randomness does not count towards the hash function
hash(s1), hash(s2), hash(s3), hash(s4)

### Batched API

In [ ]:
# Bactched API

# Cache setup
assert os.path.exists(
    "./caches/"), "Please run the script from the root directory of the project. To make sure all caches are created correctly."
cache = Cache("./caches/async_api_cache", size_limit=int(2e10))

# OpenAI API key setup
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
assert OPENAI_API_KEY is not None, "Please set the OPENAI_API_KEY environment variable"

# API setup
api_config = {
    "max_tokens": 100,
    "temperature": 0.7,
    "top_p": 1,
    "request_timeout": 45,
    "model": "gpt-3.5-turbo"
}
api = CachedOpenAIAPI(cache, api_config)

# Limiter setup
limiter = AsyncRoundRobin()
N = 4
for _ in range(N):
    limiter.add_resource(data=OPENAI_API_KEY)

prompt = "What is the capital of France?"
messages = [{"role": "user", "content": prompt}]

#response = await api.uncached_request(messages, limiter, n=1)
#print(response)

In [ ]:


# Setup batching API
batch_size = 2
bapi = BatchingAPI(batch_size, api, limiter)

In [ ]:
bapi.immediate_request(messages, n=1)

In [ ]:
print(f"Futures : {bapi.futures}")
print(f"Prompts : {bapi.prompts}")
print(f"Batches processed : {bapi.num_batches_processed}")

In [ ]:
class State:
    prompt: str


class Agent:

    @staticmethod
    async def step(state: State, api: BatchingAPI):
        # make request
        result = await api.buffered_request(state.prompt)

        # do something with the result
        # ...

        return result

## Crosswords

In [ ]:
# I'm keeping the same comments, etc everywhere, so that later it's easier to merge experiments.gameof24.py and experiments.crosswords.py (hydra?)

import asyncio
# set up logging
import logging
import os
import json
import random
import argparse

import numpy as np
import pandas as pd
from diskcache import Cache
from datetime import datetime
from collections import Counter


# TODO: Not sure if this is correct, I didn't know how else to handle the package paths
import sys
sys.path.append(os.getcwd()) # Project root!!

from async_engine.cached_api import CachedOpenAIAPI
from async_engine.round_robin_manager import AsyncRoundRobin
from async_engine.batched_api import BatchingAPI
from async_implementation.agents.crosswords import CrosswordsAgent
from async_implementation.states.crosswords import CrosswordsState
from async_implementation.resampling.resampler import Resampler
from utils import create_folder, email_notification

logger = logging.getLogger("experiments")
logger.setLevel(logging.DEBUG) # Order : debug < info < warning < error < critical
log_folder = f"logs/{datetime.now().date()}/{datetime.now().strftime('%H')}:00/crosswords/" # Folder in which logs will be saved (organized daily)
create_folder(log_folder)

# you should use the same cache for every instance of CachedOpenAIAPI
# that way we never pay for the same request twice
assert os.path.exists(
    "./caches/"), "Please run the script from the root directory of the project. To make sure all caches are created correctly."
cache = Cache("./caches/async_api_cache", size_limit=int(2e10))

# get OPENAI_API_KEY from env
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
assert OPENAI_API_KEY is not None, "Please set the OPENAI_API_KEY environment variable"

api_config = {
    "max_tokens": 100,
    "temperature": 0.7,
    "top_p": 1,
    "request_timeout": 45,
    "model": "gpt-3.5-turbo"
}

api = CachedOpenAIAPI(cache, api_config)
limiter = AsyncRoundRobin()
# ToDo, this is a bit hacky. OpenAI allows multiple parallel requests per key, so we add the same key multiple times
N = 4
for _ in range(N):
    limiter.add_resource(data=OPENAI_API_KEY)

# Set up Crosswords puzzles
path = "data/datasets/mini0505.json"
with open(path, "r") as file:
    dataset = json.load(file)

api = BatchingAPI(api, limiter, batch_size=2, timeout=10)

In [ ]:
puzzle_idx = 0

data, board_gt = dataset[puzzle_idx] # Data is the list of clues, board_gt is the ground truth  board
ans_gt = CrosswordsState.get_ans(board_gt) # Get the ground truth answers

states = []
for _ in range(2):
    states.append(CrosswordsState(data=data, board_gt=board_gt, ans_gt=ans_gt, randomness=random.randint(0, 1000)))

for state in states:
    print(CrosswordsState.render_board(state.board))
    print("\n\n")

In [ ]:
agent_coroutines = []
for state in states:
    agent_coroutines.append(CrosswordsAgent.step(state, api))
states, actions = zip(* await asyncio.gather(*agent_coroutines))

for state in states:
    print(CrosswordsState.render_board(state.board))
    print("\n\n")